In [1]:
import numpy as np
import tensorflow as tf
import strawberryfields as sf
from numba import njit

In [2]:
sf.__version__

'0.17.0'

In [3]:
def test_sf(cutoff_dim):
    eng = sf.Engine('fock', backend_options={'cutoff_dim':cutoff_dim})

    gamma1 = np.random.normal(scale=0.01) + 1j*np.random.normal(scale=0.01)
    phi1 = np.random.normal(scale=0.01)
    zeta1 = np.random.normal(scale=0.01) + 1j*np.random.normal(scale=0.01)

    
    prog = sf.Program(1)
    gamma1r = np.abs(gamma1)
    gamma1phi = np.angle(gamma1)

    with prog.context as mode:
        sf.ops.Dgate(gamma1r,phi=gamma1phi) | mode[0]
        sf.ops.Rgate(phi1) | mode[0]
        sf.ops.Sgate(zeta1) | mode[0]
    state_out = eng.run(prog).state.ket()
    
    return state_out

In [4]:
import numpy as np
from numba import jit

@njit  # (nb.types.Tuple((nb.complex128, nb.complex128[:], nb.complex128[:,:]))(nb.complex128, nb.float64, nb.complex128))
def C_mu_Sigma(gamma: np.complex, phi: np.float, z: np.complex) -> tuple:
    """
    Utility function to construct:
    1. C constant
    2. Mu vector
    3. Sigma matrix
    Arguments:
        gamma (complex): displacement parameter
        phi (float): phase rotation parameter
        z (complex): squeezing parameter
        dtype (numpy type): unused for now
    Returns:
        C (complex), mu (complex array[2]), Sigma (complex array[2,2])
    """
#     z = convert_scalar(z)
#     phi = convert_scalar(phi)
#     gamma = convert_scalar(gamma)
    r = np.abs(z)
    delta = np.angle(z)
    exp2phidelta = np.exp(1j * (2 * phi + delta))
    eiphi = np.exp(1j * phi)
    tanhr = np.tanh(r)
    coshr = np.cosh(r)
    cgamma = np.conj(gamma)

    C = np.exp(-0.5 * np.abs(gamma) ** 2 - 0.5 * cgamma ** 2 * exp2phidelta * tanhr) / np.sqrt(coshr)
    mu = np.array([cgamma * exp2phidelta * tanhr + gamma, -cgamma * eiphi / coshr,])
    Sigma = np.array([[exp2phidelta * tanhr, -eiphi / coshr], [-eiphi / coshr, -np.exp(-1j * delta) * tanhr],])

    return C, mu, Sigma


@njit
def R_matrix(gamma: np.complex, phi: np.float, z: np.complex, cutoff: int, old_state: np.array) -> np.array:
    """
    Directly constructs the transformed state recursively and exactly.

    Arguments:
        gamma (complex): displacement parameter
        phi (float): phase rotation parameter
        z (complex): squeezing parameter
        old_state (complex array[D]): State to be transformed

    Returns:
        R (complex array[D,D]): the matrix whose 1st column is the transformed state
    """

    dtype = old_state.dtype
    C, mu, Sigma = C_mu_Sigma(gamma, phi, z)

    sqrt = np.sqrt(np.arange(cutoff, dtype=dtype))

    R = np.zeros((cutoff, cutoff), dtype=dtype)
    G0 = np.zeros(cutoff, dtype=dtype)

    # first row of Transformation matrix
    G0[0] = C
    for n in range(1, cutoff):
        G0[n] = mu[1] / sqrt[n] * G0[n - 1] - Sigma[1, 1] * sqrt[n - 1] / sqrt[n] * G0[n - 2]

    # first row of R matrix
    for n in range(cutoff):
        R[0, n] = np.dot(G0[: cutoff - n], old_state)
        old_state = old_state[1:] * sqrt[1 : cutoff - n]

    # rest of R matrix
    for m in range(1, cutoff):
        for n in range(cutoff - m):
            R[m, n] = (
                mu[0] / sqrt[m] * R[m - 1, n]
                - Sigma[0, 0] * sqrt[m - 1] / sqrt[m] * R[m - 2, n]
                - Sigma[0, 1] / sqrt[m] * R[m - 1, n + 1]
            )

    return R

@jit(nopython=True)
def G_matrix(phiR, w, z, cutoff, dtype=np.complex128):
    """Calculates the Fock representation of the single-mode Gaussian gate parametrized
    as the product of the three gates R(phi)D(w)S(z).
    Args:
        phiR (float): angle of the phase rotation gate
        w (complex): displacement parameter
        z (complex): squeezing parameter
        cutoff (int): Fock ladder cutoff
        dtype (data type): Specifies the data type used for the calculation
    Returns:
        array[complex]: The Fock representation of the gate
    """

    rS = np.abs(z)
    phiS = np.angle(z)
    EZ = np.exp(1j*phiS)
    T = np.tanh(rS)
    S = 1/np.cosh(rS)
    ER = np.exp(1j*phiR)
    wc = np.conj(w)

    # 2nd derivatives of G
    R = np.array([
        [-T*EZ*ER**2, ER*S],
        [ER*S, T*np.conj(EZ)]
    ])

    # 1st derivatives of G
    y = np.array([ER*(w + T*EZ*wc), -wc*S])

    sqrt = np.sqrt(np.arange(cutoff))
    Z = np.zeros((cutoff, cutoff), dtype=dtype)

    Z[0, 0] = np.exp(-0.5*np.abs(w)**2 - 0.5*wc**2 * EZ * T)*np.sqrt(S)

    # rank 1
    for m in range(1, cutoff):
        Z[m, 0] = (y[0]/sqrt[m]*Z[m-1, 0] + R[0, 0]*sqrt[m-1]/sqrt[m]*Z[m-2, 0])

    # rank 2
    for m in range(cutoff):
        for n in range(1, cutoff):
            Z[m, n] = (y[1]/sqrt[n]*Z[m, n-1] + R[1, 1]*sqrt[n-1]/sqrt[n]*Z[m, n-2] + R[0, 1]*sqrt[m]/sqrt[n]*Z[m-1, n-1])

    return Z

In [5]:
def test_singlemode(cutoff_dim):
    gamma1 = np.random.normal(scale=0.01) + 1j*np.random.normal(scale=0.01)
    phi1 = np.random.normal(scale=0.01)
    zeta1 = np.random.normal(scale=0.01) + 1j*np.random.normal(scale=0.01)

    Psi = np.zeros(cutoff_dim,dtype=np.complex128)
    Psi[0] = 1  #vaccum
    state_out = R_matrix(gamma1,phi1, zeta1,cutoff_dim, Psi)[:,0]
    return state_out

In [6]:
def test_singlemode2(cutoff_dim):
    gamma1 = np.random.normal(scale=0.01) + 1j*np.random.normal(scale=0.01)
    phi1 = np.random.normal(scale=0.01)
    zeta1 = np.random.normal(scale=0.01) + 1j*np.random.normal(scale=0.01)

    Psi = np.zeros(cutoff_dim,dtype=np.complex128)
    Psi[0] = 1  #vaccum
    G = G_matrix(phi1, gamma1, zeta1, cutoff_dim)
    return tf.einsum("ab,b->a",G,Psi)

In [7]:
x = [5,10,15,20,25,30,35,40]

In [8]:
time_list_sf = []
for cutoff_dim in x:
    t = %timeit -o test_sf(cutoff_dim)
    time_list_sf.append(t.average)

601 µs ± 99.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
677 µs ± 57 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
741 µs ± 93.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
972 µs ± 199 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
673 µs ± 32.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1 ms ± 175 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
849 µs ± 135 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
714 µs ± 6.71 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
time_list_2mode = []
for cutoff_dim in x:
    t = %timeit -o test_singlemode(cutoff_dim)
    time_list_2mode.append(t.average)

36.9 µs ± 23.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
27.2 µs ± 469 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
29.9 µs ± 377 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
33.5 µs ± 599 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
37.8 µs ± 760 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
41.4 µs ± 365 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
46.4 µs ± 273 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
52 µs ± 597 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
time_list_2mode2 = []
for cutoff_dim in x:
    t = %timeit -o test_singlemode2(cutoff_dim)
    time_list_2mode2.append(t.average)

131 µs ± 71 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
97.4 µs ± 3.26 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
97.9 µs ± 3.48 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
104 µs ± 6.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
105 µs ± 718 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
from matplotlib import pyplot as plt
plt1 = plt.plot(x,time_list_sf,'r');
plt2 = plt.plot(x,time_list_2mode,'b');
plt3 = plt.plot(x,time_list_2mode2,'y');
plt.yscale('log')
plt.grid(True)
plt.legend((plt1[0],plt2[0],plt3[0]),("Strawberryfields __version__=0.15.1","Fast Gaussian transformation","Thewalrus/Ggate"), loc='upper left', bbox_to_anchor=(1.05, 1))

In [ ]:
time_list_sf

In [ ]:
time_list_2mode

In [ ]:
time_list_2mode2